In [1]:
%run ./MovieLens_Ranking.ipynb

In [2]:
# 设置超参数
d = 20
lambda_ = 0.01
lr = 0.01
epochs = 20
rho = 3
k = 5

In [3]:
# 求positive,negative和unobserved
positive, negative, unobserved = [], [], []
for u in range(user_num + 1):
    for j in range(item_num + 1):
        if y_ui[u][j] == 0:
            unobserved.append((u, j, -1))
        else:
            if ratings[u][j] >= r_u[u]:
                positive.append((u, j, 1))
            else:
                negative.append((u, j, -1))

def getUnion():
    union = []
    random.shuffle(unobserved)
    for i in range(len(positive)):
        union.append(positive[i])
    for i in range(len(negative)):
        union.append(negative[i])
    for i in range(rho * (len(positive) + len(negative))):
        union.append(unobserved[i])
    random.shuffle(union)
    return union

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [4]:
# 初始化参数
def init():
    mu = r_
    bu = np.zeros(user_num + 1, float)
    for u in range(1, user_num + 1):
        bu[u] = (0 if y_ui[u].sum() == 0 else (y_ui[u] * (ratings[u] - mu)).sum() / y_ui[u].sum())
    bi = np.zeros(item_num + 1, float)
    for i in range(1, item_num + 1):
        bi[i] = (0 if y_ui[:,i].sum() == 0 else (y_ui[:,i] * (ratings[:,i] - mu)).sum() / y_ui[:,i].sum())
    U = np.random.rand(user_num + 1, d)
    V = np.random.rand(item_num + 1, d)
    M = np.random.rand(item_num + 1, 6, d)
    U = (U - 0.5) * 0.01
    V = (V - 0.5) * 0.01
    M = (M - 0.5) * 0.01
    return mu, bu, bi, U, V, M

# 求用户u在M上的偏好
def U_MPC(u, M):
    m = np.zeros(d, float)
    for r in R_u(u):
        items = I_u_r(u, r)
        m += np.sum(M[items, r, :], axis=0) / np.sqrt(len(items))
    return m

# 预测函数
def predict_rule(u, i, mu, bu, bi, U, V, M):
    return U[u] @ V[i].T + U_MPC(u, M) @ V[i].T + bu[u] + bi[i]+ mu

In [5]:
# 训练
mu, bu, bi, U, V, M = init()

for epoch in tqdm(range(epochs)):
    # 求union
    union = getUnion()
    for t in range(len(union)):
        # 随机取数据
        u, i, y = union[t][0], union[t][1], union[t][2]
        # 计算梯度
        y_hat = predict_rule(u, i, mu, bu, bi, U, V, M)
        e = (sigmoid(y * y_hat) - 1) * y
        delta_mu = e
        delta_bu = e + lambda_ * bu[u]
        delta_bi = e + lambda_ * bi[i]
        delta_Uu = e * V[i] + lambda_ * U[u]
        delta_Vi = e * (U[u] + U_MPC(u, M)) + lambda_ * V[i]
        delta_M = np.zeros_like(M)
        for r in R_u(u):
            items = I_u_r(u, r)
            for item in items:
                delta_M[item, r, :] = e * V[i] / np.sqrt(len(items)) + lambda_ * M[item, r, :]
        # 更新参数
        mu -= lr * delta_mu
        bu[u] -= lr * delta_bu
        bi[i] -= lr * delta_bi
        U[u] -= lr * delta_Uu
        V[i] -= lr * delta_Vi
        for r in R_u(u):
            items = I_u_r(u, r)
            M[items, r, :] -= lr * delta_M[items, r, :]
    # 学习率下降
    lr *= 0.9

100%|██████████████████████████████████████████| 20/20 [34:18<00:00, 102.94s/it]


In [6]:
# 预测规则
def CCR(u, j):
    return predict_rule(u, j, mu, bu, bi, U, V, M)

In [7]:
# 输出结果
predict(CCR, k)

Pre: 0.2425
Rec: 0.0620
F1: 0.0826
NDCG: 0.2548
One_call: 0.6557
